In [1]:
import os
import glob
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import operator
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
os.chdir("/Users/Ruihao/Desktop/course/AML/Final/data")

train_feat, test_feat = [], []
for line in open("./features_train/features_resnet1000_train.csv"):
    l = line.strip().split(",")
    train_feat.append(l)
train_feat = np.array(train_feat)
for line in open("./features_test/features_resnet1000_test.csv"):
    l = line.strip().split(",")
    test_feat.append(l)
test_feat = np.array(test_feat)

print test_feat.shape
print train_feat.shape


(2000, 1001)
(10000, 1001)


In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_tag_dict = list()
def get_tag_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    if str not in total_tag_dict:
                        total_tag_dict.append(str)
get_tag_dict("./tags_train/")
print len(total_tag_dict)
print total_tag_dict

92
['indoor', 'teddy bear', '', 'animal', 'cat', 'electronic', 'cell phone', 'person', 'accessory', 'handbag', 'furniture', 'dining table', 'kitchen', 'knife', 'food', 'carrot', 'book', 'wine glass', 'fork', 'cake', 'chair', 'pizza', 'umbrella', 'outdoor', 'stop sign', 'sports', 'kite', 'giraffe', 'potted plant', 'appliance', 'refrigerator', 'bowl', 'apple', 'orange', 'microwave', 'oven', 'toaster', 'spoon', 'sink', 'vase', 'vehicle', 'train', 'motorcycle', 'bottle', 'bed', 'boat', 'tv', 'remote', 'zebra', 'tennis racket', 'skis', 'suitcase', 'backpack', 'snowboard', 'dog', 'banana', 'keyboard', 'mouse', 'sheep', 'broccoli', 'scissors', 'cup', 'tie', 'airplane', 'bear', 'laptop', 'clock', 'elephant', 'car', 'skateboard', 'bird', 'bench', 'surfboard', 'horse', 'truck', 'bus', 'traffic light', 'baseball bat', 'baseball glove', 'couch', 'cow', 'bicycle', 'sports ball', 'fire hydrant', 'sandwich', 'hot dog', 'toothbrush', 'donut', 'toilet', 'parking meter', 'frisbee', 'hair drier']


In [3]:
tag_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
for w in total_tag_dict:
    tag_dict[w] = index
    index += 1 
print len(tag_dict)

92


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_word_dict = {}
def get_word_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_word_dict:
                    total_word_dict[word] += 1
                else:
                    total_word_dict[word] = 1
# get the word dictionary for training description
# HashMap: (word, frequency)
get_word_dict("./descriptions_train/")
#print total_word_dict

In [5]:
word_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
# for all words in dictionary, if frequency > threshold, identify it as a high frequency word 
# assign an index to it, and increase the index
for w in total_word_dict:
    if total_word_dict[w] > 10:
        word_dict[w] = index
        index += 1 
print len(word_dict)

1804


In [6]:
def process_des(path, thresh):
    #HashMap (filename, array)
    # filename example could be 1.txt etc. 
    # array should be a 1644 length, telling us for this specific file (ex. 1.txt)
    # the frequency of all high frequency word (for example, the word yellow appears 12 times in total)
    # but in the current file, it appears 3 times
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            cur = [0] * len(total_tag_dict)
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_tag_dict:
                    cur[tag_dict[word]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec # ex. {"0.txt": [1,0,0,2,0,0,0,0,.....]; "1.txt" : [2,1,1,1,0,0,1,....]}

thresh = 10
train_des = process_des("./descriptions_train/", thresh)
print train_des["0.txt"]
print len(train_des["0.txt"])
test_des = process_des("./descriptions_test/", thresh)
#print test_des["0.txt"]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
92


In [7]:
# first part for SVM
X = []
for k in train_des:
    #train_des[k].append(k)
    X.append(train_des[k])
X = np.array(X)

print X.shape

(10000, 92)


In [8]:
def process_des(path):
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            cur = [0] * len(total_tag_dict)
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    #print str
                    if str in total_tag_dict:
                        cur[tag_dict[str]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec 

train_tag_des = process_des("./tags_train/")
print train_tag_des["0.txt"]
test_tag_des = process_des("./tags_test/")
#print test_des["0.txt"]

[0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
# second part for SVM
Y = []
for k in train_tag_des:
    #train_tag_des[k].append(k)
    Y.append(train_tag_des[k])
Y = np.array(Y)
print type(Y)
# print len(Y)
# print len(Y[0])
#print Y.shape

<type 'numpy.ndarray'>


In [10]:
print X.shape
print Y.shape
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
Y1 = [[1],[2],[3],[4]]
Y1 = np.array(Y1)

Y_sub = Y[:,0]
Y_sub = np.array(Y_sub)
print Y1.shape
print Y_sub.shape
#for i in range(91):
#     Y_sub = Y[:,i]
#     #print Y_sub  
#     X, Y_sub = make_classification(n_features=92, random_state=0)
#     clf = LinearSVC(random_state=0)
#     clf.fit(X, Y_sub)
#     p = clf.predict(test_des["100.txt"])
#     print(p)

(10000, 92)
(10000, 92)
(4, 1)
(10000,)
